In [1]:
import argparse
import os
from itertools import groupby
from hmmlearn import hmm
import gzip as gz
import pandas as pd
import numpy as np
import pickle
import plotly.express as px

In [2]:
NUM_OF_STATS = 3
STATES = {"H": 0, "S": 1, "L": 2}
AMINO_ACIDS = {"A": 0, "R": 1, "N": 2, "D": 3, "C": 4, "Q": 5, "E": 6, "G": 7, "H": 8, "I": 9,
               "L": 10, "K": 11, "M": 12, "F": 13, "P": 14, "S": 15, "T": 16, "W": 17,
               "Y": 18, "V": 19, "B": 20, "Z": 21, "X": 22, "J": 23, "O": 24, "U": 25, }
BAD_AMINO_ACIDS = "OUBZXJ"
EMISSIONS = pd.read_csv("emissions-Table 1.csv", index_col=0).T.to_numpy() / 100
TRANSITIONS = np.array([[12 / 13, 1 / 39, 2 / 39], [1 / 15, 4 / 5, 2 / 15], [0.4, 0.2, 0.4]])
PATH_TO_DATA = "./parsed_sequences" # Local

In [3]:
def train(train_samples, train_lengths, convergence_threshold: float = 0.01, num_iters: int = 80):
    # Create HMM model
    model = hmm.CategoricalHMM(n_components=NUM_OF_STATS,
                               algorithm='viterbi',
                               n_iter=num_iters,
                               tol=convergence_threshold,
                               verbose=True,
                               params="et",
                               init_params="s")

    # Config emissions
    model.emissionprob_ = EMISSIONS
    model.transmat_ = TRANSITIONS
    # Train th model
    model.fit(np.concatenate(train_samples).reshape((-1, 1)), train_lengths)

    return model

In [4]:
def split_train_test(dir_path: str):
    train_samples, train_lengths = [], []
    test_samples, test_lengths, test_labels = [], [], []

    for file in os.listdir(dir_path):
        flag = np.random.binomial(1, 0.8, 1)[0]
        file = f"{dir_path}/{file}"
        with open(file, 'r') as f:
            content = f.readlines()
            seq, labels = content[0], content[1]
            numeric_seq, numeric_labels = [], []
            for s, l in zip(seq, labels):
                if s not in BAD_AMINO_ACIDS:
                    numeric_seq += [AMINO_ACIDS[s]]
                    numeric_labels += [STATES[l]]
            if flag:
                train_samples.append(np.array(numeric_seq).reshape(-1, 1))
                train_lengths.append(len(numeric_seq))
            else:
                test_samples.append(np.array(numeric_seq).reshape(-1, 1))
                test_lengths.append(len(numeric_seq))
                test_labels.append(numeric_labels)

    return train_samples, train_lengths, test_samples, test_lengths, test_labels

In [5]:
train_samples, train_lengths, test_samples, test_lengths, test_labels = split_train_test(PATH_TO_DATA)
# model = train(train_samples, train_lengths, num_iters=67)
# # Create pickle file
# if not os.path.exists("models"):
#     os.mkdir("models")
# with open(f"models/model_bar_67", 'wb') as file:
#     pickle.dump(model, file)
model = pickle.load(open(f"./models/model_bar_67", 'rb'))

In [48]:
# Predict
misclassification_error = 0
errors_rate_per_type = {"H": 0, "S": 0, "L": 0}
num_of_acids_per_type = {"H": 0, "S": 0, "L": 0}
for sample, length, label in zip(test_samples, test_lengths, test_labels):
    ll, hidden_states = model.decode(sample, length)
    for secondary_type in errors_rate_per_type.keys():
        type_labels_indices = np.argwhere(np.array(label) == STATES[secondary_type])
        errors_rate_per_type[secondary_type] += np.count_nonzero(hidden_states[type_labels_indices] - np.array(label)[type_labels_indices])
        num_of_acids_per_type[secondary_type] += len(type_labels_indices)
# Save misclassification error
for secondary_type in STATES.keys():
    errors_rate_per_type[secondary_type] /= num_of_acids_per_type[secondary_type]
print(errors_rate_per_type)

{'H': 0.18036423774277474, 'S': 0.6097178416162305, 'L': 0.9958369778230494}


In [49]:
bar_fig = px.bar(x=errors_rate_per_type.keys(), y=errors_rate_per_type.values(), title="Error Rate per Type of Secondary Structure, 60 EM iterations")
bar_fig.update_layout(xaxis_title="Secondary Structure Type", yaxis_title="Error rate")
bar_fig.show()
if not os.path.exists("plots"):
    os.mkdir("plots")
bar_fig.write_image("plots/secondary_type.png")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [44]:
  $ pip install -U kaleido

SyntaxError: invalid syntax (3231152001.py, line 1)

In [53]:
model.emissionprob_

CategoricalHMM(init_params='s', n_components=3, n_iter=80, params='et',
               random_state=RandomState(MT19937) at 0x128652C40, verbose=True)